<a href="https://colab.research.google.com/github/aymuos/starship/blob/main/spark_rewrite/feats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Load only the first 10 records from the parquet file
df_first_10 = lazy_df.head(10).collect()

print(df_first_10)

shape: (10, 5)
┌─────┬─────────────────────────────────┬────────────────┬──────────┬──────────┐
│ ds  ┆ postman_id                      ┆ gps_time       ┆ lat      ┆ lng      │
│ --- ┆ ---                             ┆ ---            ┆ ---      ┆ ---      │
│ i64 ┆ str                             ┆ str            ┆ f64      ┆ f64      │
╞═════╪═════════════════════════════════╪════════════════╪══════════╪══════════╡
│ 321 ┆ 106f5ac22cfd1574b196d16fed62f9… ┆ 03-21 07:31:58 ┆ 3.9537e6 ┆ 3.0534e6 │
│ 321 ┆ 106f5ac22cfd1574b196d16fed62f9… ┆ 03-21 07:32:18 ┆ 3.9537e6 ┆ 3.0534e6 │
│ 321 ┆ 106f5ac22cfd1574b196d16fed62f9… ┆ 03-21 07:32:41 ┆ 3.9537e6 ┆ 3.0534e6 │
│ 321 ┆ 106f5ac22cfd1574b196d16fed62f9… ┆ 03-21 07:55:51 ┆ 3.9537e6 ┆ 3.0534e6 │
│ 321 ┆ 106f5ac22cfd1574b196d16fed62f9… ┆ 03-21 08:31:42 ┆ 3.9539e6 ┆ 3.0524e6 │
│ 321 ┆ 106f5ac22cfd1574b196d16fed62f9… ┆ 03-21 08:32:32 ┆ 3.9540e6 ┆ 3.0530e6 │
│ 321 ┆ 106f5ac22cfd1574b196d16fed62f9… ┆ 03-21 08:32:52 ┆ 3.9539e6 ┆ 3.0526e6 │
│ 321 ┆ 106f5

First, let's load the `shanghai_data.csv` file using Polars to ensure both datasets are handled consistently.

In [9]:
import polars as pl

shanghai_df = pl.read_csv("shanghai_data.csv")
print(shanghai_df.head())

shape: (5, 15)
┌─────────────┬─────────────┬─────────────┬──────────┬───┬──────────┬──────────┬─────┬─────────────┐
│ order_id    ┆ from_dipan_ ┆ delivery_us ┆ poi_lng  ┆ … ┆ sign_lng ┆ sign_lat ┆ ds  ┆ from_city_e │
│ ---         ┆ id          ┆ er_id       ┆ ---      ┆   ┆ ---      ┆ ---      ┆ --- ┆ n           │
│ str         ┆ ---         ┆ ---         ┆ f64      ┆   ┆ str      ┆ str      ┆ i64 ┆ ---         │
│             ┆ str         ┆ str         ┆          ┆   ┆          ┆          ┆     ┆ str         │
╞═════════════╪═════════════╪═════════════╪══════════╪═══╪══════════╪══════════╪═════╪═════════════╡
│ 687227b4d0c ┆ 08331170e24 ┆ 18ff78d2069 ┆ 1.0564e7 ┆ … ┆ null     ┆ null     ┆ 318 ┆ Shanghai    │
│ 733049b16cc ┆ 742ba7a3938 ┆ 125937a847f ┆          ┆   ┆          ┆          ┆     ┆             │
│ d566db6e…   ┆ f5b34ff2…   ┆ b701a9db…   ┆          ┆   ┆          ┆          ┆     ┆             │
│ ee46cae9ba2 ┆ 2129bfb99a2 ┆ 05cceaaa5db ┆ 1.0581e7 ┆ … ┆ null     ┆ null  

In [6]:
import polars as pl

# Re-initialize lazy_df from the parquet file
lazy_df = pl.scan_parquet("/content/drive/MyDrive/ml/trajectory.parquet")

# Load shanghai_data.csv and take only the first 30 rows
shanghai_df_subset = pl.read_csv("shanghai_data.csv").head(30)
shanghai_df_subset.write_csv("shanghai_top30.csv")
print("Exported shanghai_top30.csv")

# Perform an inner join with the subset of shanghai_df, using streaming for collection
joined_df_subset = lazy_df.join(
    shanghai_df_subset.lazy(), # Convert subset to a LazyFrame
    left_on="postman_id",
    right_on="delivery_user_id",
    how="inner"
).collect(engine="streaming")

print(joined_df_subset.head(12))

Exported shanghai_top30.csv
shape: (12, 19)
┌─────┬──────────────┬──────────┬──────────────┬───┬──────────┬──────────┬──────────┬──────────────┐
│ ds  ┆ postman_id   ┆ gps_time ┆ lat          ┆ … ┆ sign_lng ┆ sign_lat ┆ ds_right ┆ from_city_en │
│ --- ┆ ---          ┆ ---      ┆ ---          ┆   ┆ ---      ┆ ---      ┆ ---      ┆ ---          │
│ i64 ┆ str          ┆ str      ┆ f64          ┆   ┆ str      ┆ str      ┆ i64      ┆ str          │
╞═════╪══════════════╪══════════╪══════════════╪═══╪══════════╪══════════╪══════════╪══════════════╡
│ 328 ┆ dd648d4143da ┆ 03-28    ┆ 988917.63683 ┆ … ┆ null     ┆ null     ┆ 318      ┆ Shanghai     │
│     ┆ a61906cdb8c2 ┆ 07:31:43 ┆ 8            ┆   ┆          ┆          ┆          ┆              │
│     ┆ 83aedb…      ┆          ┆              ┆   ┆          ┆          ┆          ┆              │
│ 328 ┆ dd648d4143da ┆ 03-28    ┆ 988917.63683 ┆ … ┆ null     ┆ null     ┆ 318      ┆ Shanghai     │
│     ┆ a61906cdb8c2 ┆ 07:31:43 ┆ 8            

In [11]:
# Export joined_df_subset to a CSV file
joined_df_subset.write_csv("trajectory_top30.csv")
print("Exported joined data to trajectory_top30.csv")

Exported joined data to trajectory_top30.csv


Now, we will join `shanghai_df` with the `lazy_df` (which is scanned from `trajectory.parquet`) where `delivery_user_id` from `shanghai_df` matches `postman_id` from `lazy_df`. We'll use an inner join to get only the matching records.

In [10]:
shanghai_df.columns

['order_id',
 'from_dipan_id',
 'delivery_user_id',
 'poi_lng',
 'poi_lat',
 'aoi_id',
 'typecode',
 'receipt_time',
 'receipt_lng',
 'receipt_lat',
 'sign_time',
 'sign_lng',
 'sign_lat',
 'ds',
 'from_city_en']

In [16]:
# add eta columns to shanghai
shanghai_df = shanghai_df.with_columns(
    (pl.col("sign_time").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S", strict=False) -
     pl.col("receipt_time").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S", strict=False))
    .dt.total_minutes()
    .alias("horizon_ETA")
)

print(shanghai_df.head())

shape: (5, 16)
┌─────────────┬─────────────┬────────────┬──────────┬───┬──────────┬─────┬────────────┬────────────┐
│ order_id    ┆ from_dipan_ ┆ delivery_u ┆ poi_lng  ┆ … ┆ sign_lat ┆ ds  ┆ from_city_ ┆ horizon_ET │
│ ---         ┆ id          ┆ ser_id     ┆ ---      ┆   ┆ ---      ┆ --- ┆ en         ┆ A          │
│ str         ┆ ---         ┆ ---        ┆ f64      ┆   ┆ str      ┆ i64 ┆ ---        ┆ ---        │
│             ┆ str         ┆ str        ┆          ┆   ┆          ┆     ┆ str        ┆ i64        │
╞═════════════╪═════════════╪════════════╪══════════╪═══╪══════════╪═════╪════════════╪════════════╡
│ 687227b4d0c ┆ 08331170e24 ┆ 18ff78d206 ┆ 1.0564e7 ┆ … ┆ null     ┆ 318 ┆ Shanghai   ┆ 76         │
│ 733049b16cc ┆ 742ba7a3938 ┆ 9125937a84 ┆          ┆   ┆          ┆     ┆            ┆            │
│ d566db6e…   ┆ f5b34ff2…   ┆ 7fb701a9db ┆          ┆   ┆          ┆     ┆            ┆            │
│             ┆             ┆ …          ┆          ┆   ┆          ┆     ┆  